In [1]:
!pip install stellargraph[demos]

  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)


  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.0
    Uninstalling numpy-1.18.0:
      Successfully uninstalled numpy-1.18.0
  Rolling back uninstall of numpy
  Moving to /home/jovyan/.local/bin/f2py
   from /tmp/pip-uninstall-r_tg1ud3/f2py
  Moving to /home/jovyan/.local/bin/f2py3
   from /tmp/pip-uninstall-r_tg1ud3/f2py3
  Moving to /home/jovyan/.local/bin/f2py3.7
   from /tmp/pip-uninstall-r_tg1ud3/f2py3.7
  Moving to /home/jovyan/.local/lib/python3.7/site-packages/numpy-1.18.0.dist-info/
   from /home/jovyan/.local/lib/python3.7/site-packages/~umpy-1.18.0.dist-info
  Moving to /home/jovyan/.local/lib/python3.7/site-packages/numpy/
   from /home/jovyan/.local/lib/python3.7/site-packages/~-mpy
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/home/user/conda/lib/python3.7/site-packages/numpy/_distributor_init.py'
Consider using the `--user` option or check the permissions.



In [2]:
!git clone https://github.com/stellargraph/stellargraph/blob/develop/demos/node-classification/graphsage-inductive-node-classification.ipynb

fatal: destination path 'graphsage-inductive-node-classification.ipynb' already exists and is not an empty directory.


In [3]:
!pip install networkx

In [4]:
#!export PYTHONPATH=$PYTHONPATH:~/.local/lib/python3.7/site-packages

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 500)
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler
from sklearn.model_selection import train_test_split
import networkx as nx
from stellargraph.mapper import GraphWaveGenerator
from stellargraph import StellarGraph
from sklearn.decomposition import PCA
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse.linalg import eigs
import tensorflow as tf
from tensorflow.keras import backend as K


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score

from stellargraph import globalvar

from stellargraph import datasets
from IPython.display import display, HTML

## DATASET PROCESING

Загружаем граф:

In [120]:
graph = pd.read_csv("graph_abakan_F_encoded.csv",index_col = 0)

In [121]:
graph 

,edge_id,length,width,rm_car_speed,type,light_begin,light_end,turn_to,payments_flags,is_city_edge,barrier,turn_restrictions,traverse,passability,pedo_offset,bad_road,def_speed,road_exit,adjacent,road_classes_1,road_classes_2,road_classes_3,road_classes_4,road_classes_5,road_classes_6,road_classes_7,road_classes_8,road_classes_9,sm_classes_0,sm_classes_1,sm_classes_2,sm_classes_3,sm_classes_4,sm_classes_5,sm_classes_6,sm_classes_7,sm_classes_8,sm_classes_9,sm_classes_10,sm_classes_11,sm_classes_12,sm_classes_13,sm_classes_14,sm_classes_15,sm_classes_16,sm_classes_17,sm_classes_18,sm_classes_19,sm_classes_20,sm_classes_21,sm_classes_22,sm_classes_23,sm_classes_24,sm_classes_25,sm_classes_26,style_<pandas.io.formats.style.Styler object at 0x000001B5DF375370>,lanes_0,lanes_1,lanes_2,lanes_3,rm_car_classes_0,rm_car_classes_1,rm_car_classes_2,rm_car_classes_3,rm_car_classes_4,rm_car_classes_5,rm_car_classes_6,rm_car_classes_7,rm_car_classes_8,rm_car_classes_9,rm_car_classes_10,rm_car_classes_11,rm_car_classes_12,rm_car_classes_13,rm_car_classes_14,rm_car_classes_15
0,19423207912112130,104,7,90,0,0,0,0,0,0,0,0,1,1,0,0,90,0,"1, 64926, 64927",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,19423207912112131,104,7,90,0,0,0,0,0,0,0,0,1,1,0,0,90,0,"0, 29, 31, 28, 30",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,19423207912112132,553,3,90,0,0,0,1,0,0,0,0,1,1,0,0,90,0,"3, 44120, 44121",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,19423207912112133,553,3,90,0,0,0,1,0,0,0,0,1,1,0,0,90,0,"2, 9, 543, 8, 542",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,19423207912112134,500,7,90,0,0,0,0,0,0,0,0,1,1,0,0,90,0,"5, 7, 11, 854, 6, 10, 855",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65519,140060152502123605,91,3,60,0,0,0,1,0,1,0,0,1,1,0,0,60,0,"65518, 65521, 65520",0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
65520,140060152502123620,144,3,60,0,0,0,1,0,1,0,0,1,1,0,0,60,0,"65518, 65521, 65519",0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
65521,140060152502123621,144,3,60,0,0,0,1,0,1,0,0,1,1,0,0,60,0,"65516, 65520, 65523, 65517, 65522",0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
65522,140060152502123644,58,3,60,0,0,0,1,0,1,0,0,1,1,0,0,60,0,"65516, 65520, 65523, 65517, 65521",0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


Генерируем edge list:

In [122]:
def get_index_tuples(df):
    lst = []
    for i in tqdm(range(len(df))):
        tmp = df['adjacent'][i].split(',')
        for edge in tmp:
            lst.append([i, int(edge)])
            
    return lst
lst = get_index_tuples(graph)
lst = pd.DataFrame(lst)
lst

100%|██████████| 65524/65524 [00:01<00:00, 37620.70it/s]


,0,1
0,0,1
1,0,64926
2,0,64927
3,1,0
4,1,29
...,...,...
340007,65522,65520
340008,65522,65523
340009,65522,65517
340010,65522,65521


In [123]:
edge_data = pd.DataFrame(
    {
        "source": list(lst[0]),
        "target": list(lst[1])
    }
)
edge_data

,source,target
0,0,1
1,0,64926
2,0,64927
3,1,0
4,1,29
...,...,...
340007,65522,65520
340008,65522,65523
340009,65522,65517
340010,65522,65521


In [124]:
node_data = graph.drop(["edge_id", "adjacent", "style_<pandas.io.formats.style.Styler object at 0x000001B5DF375370>"], axis = 1)

Дропаем лишнее (структурное описание/копии индекса/артефакты): 

Создаем граф:

In [125]:
G = sg.StellarGraph(
    {"street": node_data}, {"connection": edge_data}
)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 65524, Edges: 340012

 Node types:
  street: [65524]
    Features: float32 vector, length 73
    Edge types: street-connection->street

 Edge types:
    street-connection->street: [340012]
        Weights: all 1 (default)
        Features: none


In [126]:
print("Finish?")

Finish?


In [134]:
sample_points = np.linspace(0, 100, 15).astype(np.float32)
degree = 20
scales = [5, 10]

generator = GraphWaveGenerator(G, scales=scales, degree=degree)

In [135]:
np.linspace(0, 100, 20)

array([  0.        ,   5.26315789,  10.52631579,  15.78947368,
        21.05263158,  26.31578947,  31.57894737,  36.84210526,
        42.10526316,  47.36842105,  52.63157895,  57.89473684,
        63.15789474,  68.42105263,  73.68421053,  78.94736842,
        84.21052632,  89.47368421,  94.73684211, 100.        ])

In [136]:
embeddings_dataset = generator.flow(
    node_ids=G.nodes(), sample_points=sample_points, batch_size=1, repeat=False
)

embeddings = [x.numpy() for x in embeddings_dataset]

In [137]:
len(embeddings)

65524

In [138]:
node_ = [embeddings[i][0] for i in range(len(embeddings))]

In [140]:
node_[1000]

array([1.0000000e+00, 1.0000000e+00, 9.9999696e-01, 9.9999815e-01,
       9.9998796e-01, 9.9999261e-01, 9.9997294e-01, 9.9998331e-01,
       9.9995202e-01, 9.9997038e-01, 9.9992537e-01, 9.9995381e-01,
       9.9989325e-01, 9.9993360e-01, 9.9985576e-01, 9.9990994e-01,
       9.9981314e-01, 9.9988270e-01, 9.9976569e-01, 9.9985206e-01,
       9.9971378e-01, 9.9981803e-01, 9.9965769e-01, 9.9978083e-01,
       9.9959773e-01, 9.9974030e-01, 9.9953419e-01, 9.9969679e-01,
       9.9946761e-01, 9.9965030e-01, 0.0000000e+00, 0.0000000e+00,
       1.0615781e-04, 9.2096001e-05, 2.1186048e-04, 1.8401512e-04,
       3.1665692e-04, 2.7558111e-04, 4.2010401e-04, 3.6661848e-04,
       5.2177056e-04, 4.5695333e-04, 6.2124076e-04, 5.4641359e-04,
       7.1811827e-04, 6.3482998e-04, 8.1202888e-04, 7.2203559e-04,
       9.0262428e-04, 8.0786709e-04, 9.8958414e-04, 8.9216523e-04,
       1.0726190e-03, 9.7477424e-04, 1.1514722e-03, 1.0555437e-03,
       1.2259217e-03, 1.1343277e-03, 1.2957809e-03, 1.2109866e

## CONCATENATE EMBEDDINGS

In [18]:
node_embeddings = embeddings

Загружаем файл с путями:

In [22]:
routes = pd.read_csv("abakan_full_routes_final_weather_L_NaN_filtered.csv", index_col = 0)

Дропаем лишние копии индекса (если они имеются):

In [23]:
c_routes = routes.drop(["index", "level_0","Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1"], axis = 1)
#c_routes = routes.drop(["Unnamed: 0.1","Unnamed: 0.1.1"], axis = 1)

In [24]:
c_routes

,dist_to_b,dist_to_a,start_timestamp,RTA,real_dist,pred_dist,rebuildCount,route_type,start_point_meters,finish_point_meters,start_point_part,finish_point_part,edges,time,speed,length,day_period,week_period,clouds,snow,temperature,wind,pressure
0,65,2,2020-12-14 16:00:22.0,757,2936,3495,2,traffic,1.0,25.0,0.785760,0.531041,"' 19423207912140509', ' 19423207912140505', ' ...","' 3360', ' 66600', ' 33720', ' 10500', ' 1700'...","' 15', ' 15', ' 15', ' 36', ' 36', ' 27', ' 26...","' 14', ' 90', ' 78', ' 55', ' 17', ' 147', ' 5...",2,0,1,0,-6,180,746
1,48,0,2020-12-12 10:11:27.0,501,4245,4163,2,traffic,6.0,16.0,0.722993,0.290994,"' 19423207912141719', ' 19423207912141691', ' ...","' 4800', ' 23160', ' 2640', ' 11840', ' 2700',...","' 15', ' 15', ' 15', ' 20', ' 20', ' 35', ' 40...","' 20', ' 34', ' 11', ' 38', ' 15', ' 62', ' 13...",1,0,1,0,-9,180,750
2,28,55,2020-12-06 12:21:17.0,372,3523,3514,0,traffic,5.0,25.0,0.554582,0.899702,"' 19423207912137899', ' 19423207912137895', ' ...","' 3120', ' 21000', ' 26351', ' 11793', ' 10303...","' 15', ' 15', ' 29', ' 29', ' 29', ' 29', ' 35...","' 13', ' 25', ' 172', ' 95', ' 83', ' 25', ' 1...",2,0,1,0,-21,315,752
3,81,2,2020-12-13 14:25:20.0,475,3787,4178,1,traffic,11.0,32.0,0.496820,0.434923,"' 19423207912142002', ' 19423207912142014', ' ...","' 8640', ' 8880', ' 15800', ' 960', ' 6080', '...","' 15', ' 15', ' 15', ' 15', ' 50', ' 45', ' 20...","' 36', ' 37', ' 45', ' 4', ' 15', ' 52', ' 90'...",2,0,5,2,-7,180,746
4,17,0,2020-12-04 17:43:41.0,643,4595,4672,0,traffic,1.0,10.0,0.566270,0.321134,"' 140060152264356559', ' 140060152264357499', ...","' 3840', ' 5280', ' 4080', ' 13400', ' 19926',...","' 15', ' 15', ' 15', ' 15', ' 19', ' 17', ' 17...","' 16', ' 22', ' 17', ' 35', ' 26', ' 35', ' 29...",2,0,3,0,-18,270,749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122360,72,0,2020-12-29 10:02:33.0,371,781,765,1,traffic,1.0,17.0,0.901433,0.920319,"' 19423207912137925', ' 19423207912126325', ' ...","' 960', ' 15675', ' 18822', ' 9154', ' 9360', ...","' 15', ' 29', ' 35', ' 35', ' 35', ' 15', ' 15...","' 4', ' 86', ' 183', ' 89', ' 91', ' 78', ' 90...",1,0,1,0,-26,0,758
122361,76,2,2020-12-25 16:17:11.0,329,1165,1191,2,traffic,41.0,71.0,0.429539,0.634195,"' 19423207912132093', ' 19423207912132309', ' ...","' 6640', ' 15840', ' 1440', ' 1728', ' 981', '...","' 45', ' 35', ' 20', ' 50', ' 33', ' 33', ' 40...","' 83', ' 154', ' 8', ' 24', ' 9', ' 157', ' 30...",2,0,1,0,-18,270,744
122362,54,18,2020-12-18 16:52:58.0,916,6542,6570,0,traffic,8.0,20.0,0.830191,0.582137,"' 140060152380920943', ' 19423207912122553', '...","' 2454', ' 3432', ' 12436', ' 39600', ' 7200',...","' 44', ' 43', ' 33', ' 45', ' 44', ' 49', ' 41...","' 30', ' 41', ' 114', ' 495', ' 88', ' 93', ' ...",2,0,1,0,-2,180,746
122363,30,0,2020-12-27 14:53:40.0,187,1145,1263,3,traffic,7.0,40.0,0.590148,0.460256,"' 19423207912140481', ' 19423207912140477', ' ...","' 6000', ' 10280', ' 6928', ' 385', ' 5528', '...","' 15', ' 15', ' 28', ' 28', ' 28', ' 28', ' 28...","' 25', ' 22', ' 15', ' 3', ' 43', ' 46', ' 54'...",2,1,1,0,-30,0,762


Дропаем лишние признаки:

In [25]:
X_pre = c_routes.drop(["real_dist","pred_dist","rebuildCount", "speed", "time", "length", "route_type","start_timestamp"], axis = 1) 

Проверяем индексацию:

In [26]:
X_pre = X_pre.reset_index().drop(["index"], axis = 1)

In [27]:
X_pre["edges"] = X_pre["edges"].map(lambda x: x.replace("'", ""))

In [28]:
X_pre.loc[22225, "edges"]

' 19423207912117260,  19423207912117248,  19423207912135772,  19423207912117176,  140060152264290208,  19423207912117002,  19423207912116940,  19423207912116882,  19423207912116876,  19423207912116780,  19423207912116706,  19423207912116656,  19423207912116542,  19423207912116526,  19423207912116500,  19423207912116482,  19423207912116412,  19423207912116272,  19423207912116457,  19423207912116663,  19423207912116845'

Процессим edges: (будет исправлено позднее)

In [29]:
def summ(first, second):
    return [x + y for x, y in zip(first, second)]

In [30]:
def prod(first, second):
    return [x*y for x, y in zip(first, second)]

In [31]:
def zerolistmaker(n):
    return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]

Размер эмбеддинга:

Определяем размер батча для распараллеливания построения эмбеддинга подграфа: 

In [59]:
workers_count = 27

In [176]:
emb_len = 60

In [155]:
emb_len

200

In [61]:
part_const = int(len(X_pre)/workers_count)
len(X_pre)/workers_count

4530.037037037037

In [141]:
pd.DataFrame(node_).to_csv("sage_embedding2.csv")

In [142]:
node_embeddings = pd.read_csv("sage_embedding2.csv", index_col = 0)

In [184]:
len(node_embeddings.loc[10])

60

In [177]:
import multiprocessing as mp
import psutil

def spawn():
    dataframe_list = []
    procs = list()
    n_cpus = psutil.cpu_count()
    manager = mp.Manager()
    return_list = manager.list()
    
    for cpu in range(workers_count):
        d = dict(border = part_const*(cpu+1))
        p = mp.Process(target=run_child, args=(d, return_list))
        p.start()
        procs.append(p)
    for p in procs:
        p.join()
        print('joined')
    
    return return_list

def run_child(border, return_list):
    emb = []
    for i in tqdm(range(border['border'] - part_const, border['border'])):
        path_emb = zerolistmaker(emb_len)
#        print(X_pre.loc[i, "edges"])
        arr = X_pre.loc[i, "edges"].replace("'", "").split(",")
        for j in range(len(arr)):
            ind = graph[graph["edge_id"] == int(arr[j])].index
            if (len(ind) != 0):
                ind = graph[graph["edge_id"] == int(arr[j])].index[0]
#            else:
#                print(str(i) + " " + str(j) + " " + arr[j] + " "+ str(graph[graph["edge_id"] == int(arr[j])].index[0]))
                path_emb = summ(node_embeddings.loc[ind], path_emb)
            
#                print(path_emb)                
        path_emb = [x / len(arr) for x in path_emb]
        emb.append(path_emb)
    return_list.append([(int(border['border'])-part_const)/part_const, emb])
 #   print(border['border'])
#    df = pd.DataFrame(dataframe[(int(border['border'])-part_const):border['border']]).reset_index()
#    return_list.append([(int(border['border'])-part_const)/part_const,processing.flatternize(16, filepath = "", dataframe = df)])

In [178]:
a = spawn()  

100%|█████████▉| 4523/4530 [13:03<00:00,  9.84it/s]

joined
joined
joined
joined
joined


 99%|█████████▉| 4476/4530 [13:03<00:06,  7.98it/s]

joined


100%|██████████| 4530/4530 [13:08<00:00,  5.74it/s]


joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined


In [189]:
print(len(a[0][1]))

4530


Собираем данные:

In [179]:
normal_container_s = []
for i in tqdm(range(len(a))):
    normal_container_s.append(a[i])
a_ = sorted(normal_container_s,key=lambda x: (x[0],x[1]))
a_c = a_.copy() 

100%|██████████| 27/27 [00:02<00:00, 11.09it/s]


In [180]:
a_ = sorted(normal_container_s,key=lambda x: (x[0],x[1]))

In [181]:
result = a_[0][1]
for i in range(1, len(a_)):
    result += a_[i][1]

In [183]:
result[222]

[1.0,
 1.0,
 0.9999985694885254,
 0.9999991655349731,
 0.999994158744812,
 0.9999966621398926,
 0.9999868869781494,
 0.9999925494194032,
 0.9999766349792479,
 0.99998676776886,
 0.999963641166687,
 0.9999792575836182,
 0.9999477863311768,
 0.9999702572822572,
 0.999929130077362,
 0.9999595880508424,
 0.9999077320098876,
 0.9999472498893738,
 0.9998836517333984,
 0.9999333620071412,
 0.9998569488525392,
 0.9999179840087892,
 0.9998276233673096,
 0.9999009370803832,
 0.9997959733009338,
 0.9998824000358582,
 0.9997617602348328,
 0.9998624324798584,
 0.999725341796875,
 0.9998409748077391,
 0.0,
 0.0,
 0.00010621210094541313,
 9.211714495904744e-05,
 0.00021229284175205976,
 0.0001841829507611692,
 0.00031811147346161306,
 0.00027614616556093097,
 0.0004235382657498121,
 0.00036795582855120296,
 0.0005284450598992407,
 0.00045956138637848204,
 0.0006327060982584952,
 0.0005509127513505518,
 0.0007361980970017612,
 0.0006419605342671275,
 0.0008388009155169129,
 0.0007326561608351767,
 0.0

In [182]:
len(result[222])

50

Технические проверки:

In [157]:
X_pre_c = X_pre.drop(X_pre[X_pre["snow"] == -1].index)

In [158]:
set(X_pre_c["day_period"])

{0, 1, 2, 3}

Объединяем данные путей с эмбеддингами, завершаем подготовку данных к обучению:

In [159]:
X = X_pre_c.join(pd.DataFrame(result)).dropna().drop(["edges"], axis = 1).reset_index(drop=True)

In [160]:
X

,dist_to_b,dist_to_a,RTA,start_point_meters,finish_point_meters,start_point_part,finish_point_part,day_period,week_period,clouds,snow,temperature,wind,pressure,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,65,2,757,1.0,25.0,0.785760,0.531041,2,0,1,0,-6,180,746,1.0,1.0,0.999998,0.999999,0.999993,0.999996,0.999984,0.999991,0.999972,0.999984,0.999956,0.999975,0.999937,0.999964,0.999915,0.999951,0.999889,0.999936,0.999861,0.999919,0.999830,0.999900,0.999796,0.999880,0.999760,0.999858,0.999722,0.999834,0.999681,0.999808,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000423,0.000368,0.000526,0.000459,0.000629,0.000550,0.000731,0.000640,0.000831,0.000730,0.000929,0.000819
1,48,0,501,6.0,16.0,0.722993,0.290994,1,0,1,0,-9,180,750,1.0,1.0,0.999999,0.999999,0.999996,0.999998,0.999991,0.999995,0.999985,0.999991,0.999976,0.999986,0.999965,0.999980,0.999953,0.999973,0.999939,0.999965,0.999922,0.999955,0.999904,0.999945,0.999884,0.999933,0.999863,0.999921,0.999839,0.999907,0.999814,0.999892,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000530,0.000460,0.000636,0.000552,0.000741,0.000644,0.000846,0.000735,0.000950,0.000827
2,28,55,372,5.0,25.0,0.554582,0.899702,2,0,1,0,-21,315,752,1.0,1.0,0.999999,0.999999,0.999996,0.999998,0.999991,0.999995,0.999984,0.999991,0.999975,0.999985,0.999964,0.999979,0.999950,0.999971,0.999935,0.999963,0.999918,0.999953,0.999899,0.999942,0.999878,0.999929,0.999855,0.999916,0.999831,0.999902,0.999804,0.999886,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000530,0.000460,0.000635,0.000552,0.000740,0.000644,0.000845,0.000735,0.000949,0.000827
3,81,2,475,11.0,32.0,0.496820,0.434923,2,0,5,2,-7,180,746,1.0,1.0,0.999998,0.999999,0.999992,0.999995,0.999982,0.999990,0.999968,0.999982,0.999950,0.999971,0.999929,0.999959,0.999903,0.999944,0.999875,0.999927,0.999842,0.999907,0.999806,0.999886,0.999767,0.999863,0.999725,0.999837,0.999680,0.999809,0.999633,0.999780,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000422,0.000368,0.000526,0.000459,0.000629,0.000550,0.000731,0.000640,0.000831,0.000730,0.000929,0.000819
4,17,0,643,1.0,10.0,0.566270,0.321134,2,0,3,0,-18,270,749,1.0,1.0,0.999998,0.999999,0.999993,0.999996,0.999984,0.999991,0.999972,0.999983,0.999956,0.999974,0.999936,0.999963,0.999914,0.999950,0.999888,0.999934,0.999859,0.999917,0.999826,0.999898,0.999791,0.999877,0.999753,0.999854,0.999713,0.999829,0.999670,0.999802,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000423,0.000368,0.000527,0.000459,0.000631,0.000550,0.000733,0.000641,0.000834,0.000731,0.000933,0.000821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117800,44,0,161,4.0,16.0,0.688172,0.663870,0,0,5,2,-7,180,746,1.0,1.0,0.999999,0.999999,0.999995,0.999997,0.999989,0.999994,0.999980,0.999989,0.999969,0.999982,0.999955,0.999974,0.999939,0.999965,0.999921,0.999955,0.999900,0.999943,0.999877,0.999929,0.999852,0.999914,0.999825,0.999898,0.999795,0.999881,0.999764,0.999862,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000529,0.000460,0.000634,0.000552,0.000738,0.000643,0.000842,0.000734,0.000945,0.000825
117801,72,0,371,1.0,17.0,0.901433,0.920319,1,0,1,0,-26,0,758,1.0,1.0,0.999999,0.999999,0.999995,0.999998,0.999989,0.999995,0.999981,0.999990,0.999971,0.999985,0.999958,0.999978,0.999943,0.999970,0.999926,0.999961,0.999906,0.999951,0.999884,0.999940,0.999860,0.999927,0.999834,0.999913,0.999806,0.999898,0.999776,0.999882,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000529,0.000460,0.000634,0.000552,0.000739,0.000644,0.000843,0.000735,0.000946,0.000827
117802,76,2,329,41

In [161]:
wind_dir_classes = pd.get_dummies(X.wind, prefix='wind_dir_class')
day_classes = pd.get_dummies(X.day_period, prefix='day_class')
Y = X["RTA"]
X = X.join(wind_dir_classes).join(day_classes).drop(["wind", "day_period", "RTA"], axis = 1)


In [162]:
X

,dist_to_b,dist_to_a,start_point_meters,finish_point_meters,start_point_part,finish_point_part,week_period,clouds,snow,temperature,pressure,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,wind_dir_class_0,wind_dir_class_45,wind_dir_class_90,wind_dir_class_135,wind_dir_class_180,wind_dir_class_225,wind_dir_class_270,wind_dir_class_315,day_class_0,day_class_1,day_class_2,day_class_3
0,65,2,1.0,25.0,0.785760,0.531041,0,1,0,-6,746,1.0,1.0,0.999998,0.999999,0.999993,0.999996,0.999984,0.999991,0.999972,0.999984,0.999956,0.999975,0.999937,0.999964,0.999915,0.999951,0.999889,0.999936,0.999861,0.999919,0.999830,0.999900,0.999796,0.999880,0.999760,0.999858,0.999722,0.999834,0.999681,0.999808,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000423,0.000368,0.000526,0.000459,0.000629,0.000550,0.000731,0.000640,0.000831,0.000730,0.000929,0.000819,0,0,0,0,1,0,0,0,0,0,1,0
1,48,0,6.0,16.0,0.722993,0.290994,0,1,0,-9,750,1.0,1.0,0.999999,0.999999,0.999996,0.999998,0.999991,0.999995,0.999985,0.999991,0.999976,0.999986,0.999965,0.999980,0.999953,0.999973,0.999939,0.999965,0.999922,0.999955,0.999904,0.999945,0.999884,0.999933,0.999863,0.999921,0.999839,0.999907,0.999814,0.999892,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000530,0.000460,0.000636,0.000552,0.000741,0.000644,0.000846,0.000735,0.000950,0.000827,0,0,0,0,1,0,0,0,0,1,0,0
2,28,55,5.0,25.0,0.554582,0.899702,0,1,0,-21,752,1.0,1.0,0.999999,0.999999,0.999996,0.999998,0.999991,0.999995,0.999984,0.999991,0.999975,0.999985,0.999964,0.999979,0.999950,0.999971,0.999935,0.999963,0.999918,0.999953,0.999899,0.999942,0.999878,0.999929,0.999855,0.999916,0.999831,0.999902,0.999804,0.999886,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000530,0.000460,0.000635,0.000552,0.000740,0.000644,0.000845,0.000735,0.000949,0.000827,0,0,0,0,0,0,0,1,0,0,1,0
3,81,2,11.0,32.0,0.496820,0.434923,0,5,2,-7,746,1.0,1.0,0.999998,0.999999,0.999992,0.999995,0.999982,0.999990,0.999968,0.999982,0.999950,0.999971,0.999929,0.999959,0.999903,0.999944,0.999875,0.999927,0.999842,0.999907,0.999806,0.999886,0.999767,0.999863,0.999725,0.999837,0.999680,0.999809,0.999633,0.999780,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000422,0.000368,0.000526,0.000459,0.000629,0.000550,0.000731,0.000640,0.000831,0.000730,0.000929,0.000819,0,0,0,0,1,0,0,0,0,0,1,0
4,17,0,1.0,10.0,0.566270,0.321134,0,3,0,-18,749,1.0,1.0,0.999998,0.999999,0.999993,0.999996,0.999984,0.999991,0.999972,0.999983,0.999956,0.999974,0.999936,0.999963,0.999914,0.999950,0.999888,0.999934,0.999859,0.999917,0.999826,0.999898,0.999791,0.999877,0.999753,0.999854,0.999713,0.999829,0.999670,0.999802,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000423,0.000368,0.000527,0.000459,0.000631,0.000550,0.000733,0.000641,0.000834,0.000731,0.000933,0.000821,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117800,44,0,4.0,16.0,0.688172,0.663870,0,5,2,-7,746,1.0,1.0,0.999999,0.999999,0.999995,0.999997,0.999989,0.999994,0.999980,0.999989,0.999969,0.999982,0.999955,0.999974,0.999939,0.999965,0.999921,0.999955,0.999900,0.999943,0.999877,0.999929,0.999852,0.999914,0.999825,0.999898,0.999795,0.999881,0.999764,0.999862,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000529,0.000460,0.000634,0.000552,0.000738,0.000643,0.000842,0.000734,0.000945,0.000825,0,0,0,0,1,0,0,0,1,0,0,0
117801,72,0,1.0,17.0,0.901433,0.920319,0,1,0,-26,758,1.0,1.0,0.999999,0.999999,0.999995,0.999998,0.999989,0.999995,0.999981,0.999990,0.999971,0.999985,0.999958,0.999978,0.999943,0.999970,0.999926,0.999961,0.999906

In [163]:
X

,dist_to_b,dist_to_a,start_point_meters,finish_point_meters,start_point_part,finish_point_part,week_period,clouds,snow,temperature,pressure,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,wind_dir_class_0,wind_dir_class_45,wind_dir_class_90,wind_dir_class_135,wind_dir_class_180,wind_dir_class_225,wind_dir_class_270,wind_dir_class_315,day_class_0,day_class_1,day_class_2,day_class_3
0,65,2,1.0,25.0,0.785760,0.531041,0,1,0,-6,746,1.0,1.0,0.999998,0.999999,0.999993,0.999996,0.999984,0.999991,0.999972,0.999984,0.999956,0.999975,0.999937,0.999964,0.999915,0.999951,0.999889,0.999936,0.999861,0.999919,0.999830,0.999900,0.999796,0.999880,0.999760,0.999858,0.999722,0.999834,0.999681,0.999808,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000423,0.000368,0.000526,0.000459,0.000629,0.000550,0.000731,0.000640,0.000831,0.000730,0.000929,0.000819,0,0,0,0,1,0,0,0,0,0,1,0
1,48,0,6.0,16.0,0.722993,0.290994,0,1,0,-9,750,1.0,1.0,0.999999,0.999999,0.999996,0.999998,0.999991,0.999995,0.999985,0.999991,0.999976,0.999986,0.999965,0.999980,0.999953,0.999973,0.999939,0.999965,0.999922,0.999955,0.999904,0.999945,0.999884,0.999933,0.999863,0.999921,0.999839,0.999907,0.999814,0.999892,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000530,0.000460,0.000636,0.000552,0.000741,0.000644,0.000846,0.000735,0.000950,0.000827,0,0,0,0,1,0,0,0,0,1,0,0
2,28,55,5.0,25.0,0.554582,0.899702,0,1,0,-21,752,1.0,1.0,0.999999,0.999999,0.999996,0.999998,0.999991,0.999995,0.999984,0.999991,0.999975,0.999985,0.999964,0.999979,0.999950,0.999971,0.999935,0.999963,0.999918,0.999953,0.999899,0.999942,0.999878,0.999929,0.999855,0.999916,0.999831,0.999902,0.999804,0.999886,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000530,0.000460,0.000635,0.000552,0.000740,0.000644,0.000845,0.000735,0.000949,0.000827,0,0,0,0,0,0,0,1,0,0,1,0
3,81,2,11.0,32.0,0.496820,0.434923,0,5,2,-7,746,1.0,1.0,0.999998,0.999999,0.999992,0.999995,0.999982,0.999990,0.999968,0.999982,0.999950,0.999971,0.999929,0.999959,0.999903,0.999944,0.999875,0.999927,0.999842,0.999907,0.999806,0.999886,0.999767,0.999863,0.999725,0.999837,0.999680,0.999809,0.999633,0.999780,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000422,0.000368,0.000526,0.000459,0.000629,0.000550,0.000731,0.000640,0.000831,0.000730,0.000929,0.000819,0,0,0,0,1,0,0,0,0,0,1,0
4,17,0,1.0,10.0,0.566270,0.321134,0,3,0,-18,749,1.0,1.0,0.999998,0.999999,0.999993,0.999996,0.999984,0.999991,0.999972,0.999983,0.999956,0.999974,0.999936,0.999963,0.999914,0.999950,0.999888,0.999934,0.999859,0.999917,0.999826,0.999898,0.999791,0.999877,0.999753,0.999854,0.999713,0.999829,0.999670,0.999802,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000423,0.000368,0.000527,0.000459,0.000631,0.000550,0.000733,0.000641,0.000834,0.000731,0.000933,0.000821,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117800,44,0,4.0,16.0,0.688172,0.663870,0,5,2,-7,746,1.0,1.0,0.999999,0.999999,0.999995,0.999997,0.999989,0.999994,0.999980,0.999989,0.999969,0.999982,0.999955,0.999974,0.999939,0.999965,0.999921,0.999955,0.999900,0.999943,0.999877,0.999929,0.999852,0.999914,0.999825,0.999898,0.999795,0.999881,0.999764,0.999862,0.0,0.0,0.000106,0.000092,0.000212,0.000184,0.000318,0.000276,0.000424,0.000368,0.000529,0.000460,0.000634,0.000552,0.000738,0.000643,0.000842,0.000734,0.000945,0.000825,0,0,0,0,1,0,0,0,1,0,0,0
117801,72,0,1.0,17.0,0.901433,0.920319,0,1,0,-26,758,1.0,1.0,0.999999,0.999999,0.999995,0.999998,0.999989,0.999995,0.999981,0.999990,0.999971,0.999985,0.999958,0.999978,0.999943,0.999970,0.999926,0.999961,0.999906

Модель #1:

In [164]:
from sklearn.linear_model import ElasticNetCV
model = ElasticNetCV()

In [110]:
prediction = model.fit(X.iloc[0:700000, :], list(Y[0:700000]))

/home/jovyan/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [111]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
a_RMSE = mean_squared_error(Y[700000:740000], prediction.predict(X.iloc[700000:740000, :]))
a_MAE = mean_absolute_error(Y[700000:740000], prediction.predict(X.iloc[700000:740000, :]))
pd.DataFrame([[a_RMSE, a_MAE]], index = ["Abakan"], columns = ["RMSE", "MAE"])

,RMSE,MAE
Abakan,123995.330585,176.792862


Модель #2:

In [165]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [166]:
modelNN = tf.keras.models.Sequential()
modelNN.add(tf.keras.Input(shape=(72,)))
modelNN.add(tf.keras.layers.Dense(128, activation='relu'))
modelNN.add(tf.keras.layers.Dense(128*2, activation='relu'))
modelNN.add(tf.keras.layers.Dense(128*4, activation='relu'))
modelNN.add(tf.keras.layers.Dense(1))

In [167]:
modelNN.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [168]:
len(X)

117805

In [169]:
train_x = np.asarray(X.iloc[0:100000, 0:72])
train_y = np.asarray(Y[0:100000])

In [170]:
test_x = np.asarray(X.iloc[100000:140000, 0:72])
test_y = np.asarray(Y[100000:140000])

In [171]:
from  tensorflow.keras.utils import normalize, to_categorical
train_x = normalize(train_x, axis=1)
test_x =  normalize(test_x, axis=1)

In [172]:
historyNN = modelNN.fit(train_x, train_y, epochs = 10)

Epoch 1/10
3125/3125 [==============================] - 11s 3ms/step - loss: 258.8839
Epoch 2/10
3125/3125 [==============================] - 9s 3ms/step - loss: 245.6181
Epoch 3/10
3125/3125 [==============================] - 10s 3ms/step - loss: 247.3226
Epoch 4/10
3125/3125 [==============================] - 10s 3ms/step - loss: 246.2393
Epoch 5/10
3125/3125 [==============================] - 10s 3ms/step - loss: 245.9470
Epoch 6/10
3125/3125 [==============================] - 10s 3ms/step - loss: 249.3790
Epoch 7/10
3125/3125 [==============================] - 9s 3ms/step - loss: 248.1564
Epoch 8/10
3125/3125 [==============================] - 10s 3ms/step - loss: 246.0190
Epoch 9/10
3125/3125 [==============================] - 9s 3ms/step - loss: 246.3004
Epoch 10/10
3125/3125 [==============================] - 9s 3ms/step - loss: 248.7481


In [173]:
result = modelNN.evaluate(
    test_x,
    test_y, verbose=0)

In [174]:
result

244.9971923828125

In [117]:
test_x

array([[0.08412627, 0.        , 0.00133534, ..., 0.        , 0.        ,
        0.00133534],
       [0.07878444, 0.        , 0.00393922, ..., 0.        , 0.00131307,
        0.        ],
       [0.0698243 , 0.05928478, 0.00131744, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.10098247, 0.00265743, 0.05447739, ..., 0.        , 0.        ,
        0.00132872],
       [0.07214263, 0.02404754, 0.0106878 , ..., 0.        , 0.        ,
        0.00133597],
       [0.03925184, 0.        , 0.00915876, ..., 0.        , 0.        ,
        0.00130839]])